## 1 Interfacing Between pandas and Model Code

A common workflow for model development is to use pandas for data loading and cleaning before switching over to a modeling library to build the model itself.

The point of contact between pandas and other analysis libraries is usually NumPy arrays. To turn a DataFrame into a NumPy array, use the `values` property

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
data = pd.DataFrame({'x0': range(1, 6),
                     'x1': [0.01, -0.01, 0.25, -4.1, 0.],
                     'y': [-1.5, 0., 3.6, 1.3, -2.]})

In [4]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [5]:
data.columns

Index([u'x0', u'x1', u'y'], dtype='object')

In [6]:
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

To convert back to a DataFrame, you can pass a two-dimensional ndarray with optional colum names

In [7]:
df2 = pd.DataFrame(data.values, columns=['one', 'two', 'trhee'])

In [8]:
df2

,one,two,trhee
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


The `.values` attribute is intended to be used when your data is homogeneous. If the data is heterogeneous, the result will be an ndarray of Python objects

In [9]:
df3 = data.copy()

In [10]:
df3['strings'] = ['a', 'b', 'c', 'd', 'e']

In [11]:
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [12]:
df3.values

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

Using `loc` to indexing with values

In [14]:
data.loc[:, ['x0', 'x1']].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

Prepare "metadata management" manually. Handling a dataset with a non-numeric column

In [18]:
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'])

In [19]:
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


Replace the 'category' column with dummy variables

In [20]:
dummies = pd.get_dummies(data.category, prefix='category')

In [21]:
data_with_dummies = data.drop('category', axis=1).join(dummies)

In [22]:
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


## 2 Creating Model Descriptions with Patsy

***Patsy*** is a Python library for describing statistical models (especially linear models) with a small string-based "formula syntax", which is well supported for specifying linear models in statsmodels.